In [ ]:
import numpy as np
from skimage import io

from matplotlib import pyplot as plt

from matplotlib import cm
import networkx as nx
from tqdm.notebook import tqdm

from bresenham import bresenham
from matplotlib import cm, colors
import matplotlib.patches as mpatches
from collections import Counter
import pandas as pd

import matplotlib.pyplot as plt

import itertools

from mpire import WorkerPool

import shutil
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pathlib import Path
import json

from wc_cv.src.utils import grainPreprocess, grainMark, SEMDataset, grainDraw, Crack

from pathos.multiprocessing import ProcessingPool as Pool  # Alternative parallelization
import copy

import graph_tool.all as gt
from itertools import islice

In [ ]:
plt.imshow(image,cmap='gray')
plt.xlabel('width, pixels',fontsize=fontsize)
plt.ylabel('height, pixels',fontsize=fontsize)
# plt.savefig('wc_cv/cv/Ultra_Co6_2-001_cut.jpeg',bbox_inches='tight')
plt.show()

In [ ]:
eps = 300

# 1 
# image = io.imread('wc_cv/cv/plots/WC+8Co_5_crack.jpg')[200:-300][:400]
# 2
# image = io.imread('wc_cv/cv/plots/WC+8Co_5_fixed_001_cropped.jpg')
# 3
# image = io.imread('wc_cv/datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-003.jpeg')[:300,:500]
# 4
image = io.imread('wc_cv/datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-001.jpeg')[:200,:500]

border=30

(entry_nodes,
 exit_nodes,
 img_contours_o,
 img_preprocessed_final,
 cnts,
 nodes_metadata)  = Crack.preprocess_graph_image(image, border=border, disk=5,entry_ellps_w=5,exit_ellps_w=5,r=4)

# plot=False
plot=True

if plot:
    fontsize=15
    # fig, axes = plt.subplots(1,2,figsize=(20,20))
    # axes[0].imshow(img_contours_o)
    # axes[1].imshow(img_preprocessed_final)

    # axes[0].set_xlabel('width, pixels',fontsize=fontsize)
    # axes[0].set_ylabel('height, pixels',fontsize=fontsize)

    # axes[1].set_xlabel('width, pixels',fontsize=fontsize)
    # axes[1].set_ylabel('height, pixels',fontsize=fontsize)

    plt.imshow(img_contours_o)
    plt.xlabel('width, pixels',fontsize=fontsize)
    plt.ylabel('height, pixels',fontsize=fontsize)
    # plt.savefig('wc_cv/cv/Ultra_Co6_2-001_cut_step_1.jpeg',bbox_inches='tight')
    plt.show()

    plt.imshow(img_preprocessed_final)
    plt.xlabel('width, pixels',fontsize=fontsize)
    plt.ylabel('height, pixels',fontsize=fontsize)
    # plt.savefig('wc_cv/cv/Ultra_Co6_2-001_cut_step_2.jpeg',bbox_inches='tight')
    plt.show()

g, img_contours =  Crack.create_crack_graph(img_preprocessed_final.shape, cnts, nodes_metadata, eps=eps)

In [ ]:
def graph_plot( g, img_contours,N=50,M=50, name ='graph.jpg', border = 30, save=False):
    # 0 - Co
    # 1 - WC-Co
    # 2 - WC
    # 3 - WC-WC

    color_dict={0:'red',
                1:'orange',
                2:'blue',
                3:'green'}
    

    colors=[]
    for u,v in g.edges():
        edge_type = g[u][v]['edge_type']
        colors.append(color_dict[edge_type])

    img_tmp = np.array(img_contours)
    
    pos = nx.get_node_attributes(g, 'pos')
    fig, axes = plt.subplots(1,1,figsize=(N,M))
    
    nx.draw(g, pos, ax = axes,  with_labels=True, node_color='lightblue', node_size=500, font_size=15, edge_color=colors)
    axes.imshow(img_tmp, cmap='gray')
    
    # axes[0].invert_yaxis()
    # axes[1].invert_yaxis()
    
    plt.axis("on")
    axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
    plt.xticks(np.arange(0,img_tmp.shape[1],100), fontsize=15)
    plt.yticks(np.arange(0,img_tmp.shape[0],100),fontsize=15)
    
    # axes.arrow(5, 0, 5, img_tmp.shape[0], width=0.3, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))
    # axes.text(-20, img_tmp.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=25)
    
    # axes.text(img_tmp.shape[1]/2 - border, - 2*border,'entry nodes', color=(0, 0, 1),fontsize=25)
    # axes.text(img_tmp.shape[1]/2 - border,img_tmp.shape[0]+ 2*border,'exit nodes', color=(1, 0, 0),fontsize=25)

    path1 = mpatches.Patch(color='red', label='Co, type 0')
    path2 = mpatches.Patch(color='orange', label='WC-Co, type 1')
    path3 = mpatches.Patch(color='blue', label='WC, type 2')
    path4 = mpatches.Patch(color='green', label='WC-WC, type 3')
    
    plt.legend(loc="upper right", handles=[path1,path2,path3,path4],fontsize=25)

    plt.xlabel('width, pixels',fontsize=25)
    plt.ylabel('height, pixels',fontsize=25)


    if save:
        plt.savefig(name, bbox_inches='tight')
        
    plt.show()

save=False
# save=True

# Crack.Viz.graph_plot(g, img_preprocessed_final,N=20,M=20, name='test.jpg', save=save)
graph_plot(g, img_preprocessed_final,N=15,M=15, name='wc_cv/cv/Ultra_Co6_2-001_cut_graph.jpeg', save=save)

In [ ]:
%%time

# 1
# entry_nodes = [ 0,1,4, 10, 16]
# exit_nodes = [ 120, 121, 31, 32, 113]

# 4
entry_nodes = [ 0,1, 3, 9, 10, 13]
exit_nodes = [ 23, 24, 71, 72, 63, 64, 56, 57, 67, 68]

# entry_nodes = [ 0,3,4, 11]
# exit_nodes = [ 84, 30, 31]

# WC+8Co_5_fixed_001_cropped.jpg
# entry_nodes = [ 4, 15, 13, 16, 34, 27]
# exit_nodes = [ 527, 528, 519, 522]

# WC+8Co_5_crack
# entry_nodes = [ 4, 11, 16, 61, 62, 66]
# exit_nodes = [ 307, 308, 515, 529]

# edge types
# 0 - Co
# 1 - WC-Co
# 2 - WC
# 3 - WC-WC


first_k_paths=1

parallel=True
# parallel=False
workers=20

param_1_max=20 # Co
param_2_max=20 # WC-Co
param_3_const=20 # WC
param_4_const=0 # WC-WC

energy_conf=np.zeros((param_1_max, param_2_max)).tolist()

for i,en_1 in enumerate(tqdm(range(0,param_1_max))):
    for j,en_2 in enumerate(range(0,param_2_max)):

        energy_conf[i][j]={
            0: en_1, # Co
            1: en_2, # WC-Co
            2: param_3_const, # WC
            3: param_4_const # WC-WC
            } 

energies_paths = Crack.Energy.get_energies(energy_conf,
                                            g,
                                            cnts,
                                            nodes_metadata,
                                            entry_nodes,
                                            exit_nodes,
                                            first_k_paths=first_k_paths,
                                            parallel=parallel,
                                            workers=workers)


In [ ]:
# name='wc_cv/cv/WC+8Co_5_fixed_001_cropped_optimized_e.jpg'
name='wc_cv/cv/WC+8Co_5_crack_fixed_optimized_e_labeled_no_wc_10.jpg'

# save=True
save=False

path_index=0

Crack.Viz.plot_optimized_energies(  energies_paths,
# plot_optimized_energies(  energies_paths,
                                    path_index=path_index,
                                    N=5,M=5,
                                    y_label ='co_e',
                                    x_label = 'wc-co_e',
                                    name=name,
                                    fontsize_h=10,
                                    fontsize_axes=50
                                    )

In [ ]:
param_1=10
param_2=10

# paths = [energies_paths[i][param_1][param_2]['path'].values[0] for i in range(len(energies_paths))]
paths = [energies_paths[i][param_1][param_2] for i in range(len(energies_paths))]
paths = pd.concat(paths, axis=0)

fig,axes = plt.subplots(1,1,figsize=(15,15))

color_dict={0:'red',
            1:'orange',
            2:'blue',
            3:'green'}

pos = nx.get_node_attributes(g, 'pos')

entry_nodes = np.unique(paths['entry_node'])
exit_nodes = np.unique(paths['exit_node'])

for entry_node in entry_nodes:

    path = paths[paths['entry_node']==entry_node].sort_values(by='path_len_pixels').iloc[0]['path']

    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges, edge_color='blue', width=2)
    nx.draw_networkx_nodes(g,pos,nodelist=[path[0]],node_color='blue',node_size = 100)
    nx.draw_networkx_nodes(g,pos,nodelist=[path[-1]],node_color='red',node_size = 100)


for exit_node in exit_nodes:

    path = paths[paths['exit_node']==exit_node].sort_values(by='path_len_pixels').iloc[0]['path']
    path = list(reversed(path)) 

    path_edges = list(zip(path,path[1:]))
    
    # colors=[]
    # for u,v in path_edges:
    #     edge_type = g[u][v]['edge_type']
    #     colors.append(color_dict[edge_type])

    nx.draw_networkx_edges(g,pos,edgelist=path_edges, edge_color='red', width=2)
    nx.draw_networkx_nodes(g,pos,nodelist=[path[0]],node_color='blue',node_size = 100)
    nx.draw_networkx_nodes(g,pos,nodelist=[path[-1]],node_color='red',node_size = 100)


axes.imshow(img_contours_o)

plt.axis("on")
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig('wc_cv/cv/shortests_paths.jpeg',bbox_inches='tight')

plt.show()

# Fixed paths energies 

In [ ]:
%%time
# df['index'] = range(len(df))
# df.set_index('index')

# WC+8Co_5_crack
# entry_nodes = [ 4, 11, 16, 61, 62, 66]
# exit_nodes = [ 307, 308, 515, 529]

# WC+8Co_5_fixed_001_cropped.jpg
entry_nodes = [ 4, 15, 13, 16, 34, 27]
exit_nodes = [ 527, 528, 519, 522]

workers=23
np.random.seed(51)
first_k_paths=1
parallel=True

#################################
# calculate base paths energies # 
#################################

param_1_const=15 # Co
param_2_const=15 # WC-Co
param_3_const=20 # WC
param_4_const=0 # WC-WC

energy_conf=np.zeros((1, 1)).tolist()

energy_conf[0][0]={
    0: param_1_const, # Co
    1: param_2_const, # WC-Co
    2: param_3_const, # WC
    3: param_4_const # WC-WC
    } 

energies_paths = Crack.Energy.get_energies( energy_conf,
# energies_paths = get_energies( energy_conf,
                                            g,
                                            cnts,
                                            nodes_metadata,
                                            entry_nodes,
                                            exit_nodes,
                                            first_k_paths=first_k_paths,
                                            parallel=parallel,
                                            workers=workers)

########################
# recalculate energies # 
########################

param_1_max= 20 # Co
param_2_max= 20 # WC-Co
param_3_const=20 # WC
param_4_const=0 # WC-WC

parallel=False

energy_conf=np.zeros((param_1_max, param_2_max)).tolist()

for i,en_1 in enumerate(tqdm(range(0,param_1_max))):
    for j,en_2 in enumerate(range(0,param_2_max)):

        energy_conf[i][j]={
            0: en_1, # Co
            1: en_2, # WC-Co
            2: param_3_const, # WC
            3: param_4_const # WC-WC
            } 

energies_paths_recalculated = Crack.Energy.get_energies( energy_conf,
# energies_paths_recalculated =    get_energies( energy_conf,
                                            g,
                                            cnts,
                                            nodes_metadata,
                                            entry_nodes,
                                            exit_nodes,
                                            first_k_paths=first_k_paths,
                                            parallel=parallel,
                                            workers=workers,
                                            recalculate_paths=energies_paths)

In [ ]:
# name='wc_cv/cv/WC+8Co_5_fixed_001_cropped_optimized_e.jpg'
name='wc_cv/cv/WC+8Co_5_crack_fixed_optimized_e_labeled_no_wc_10.jpg'

# save=True
save=False

path_index=0

Crack.Viz.plot_optimized_energies(  energies_paths_recalculated,
                                    path_index=path_index,
                                    N=5,M=5,
                                    y_label ='co_co_e',
                                    x_label = 'wc_co_e',
                                    name=name,
                                    save=save,
                                    fixed_paths=True
                                    )

# Paths visualization

In [ ]:
# plot paths with colorbar color indices

fig, axes = plt.subplots(1,1,figsize=(15,20))
border=30

pos = nx.get_node_attributes(g, 'pos')

df_shortest_entry = copy.copy(df)

x=329
y=33

# df_shortest_entry=df_shortest_entry[df_shortest_entry['path'].apply(lambda arr: x in arr)]
# df_shortest_entry=df_shortest_entry[df_shortest_entry['path'].apply(lambda arr: y in arr)]

cmap = cm.get_cmap("Spectral")
# need to normalize because color maps are defined in [0, 1]
norm = colors.Normalize(df_shortest_entry['path_len_pixel'].values.min(), df_shortest_entry['path_len_pixel'].values.max())


for i in range(len(df_shortest_entry)):
    line = df_shortest_entry.iloc[i]
    path = line['path']
    
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color=cmap(norm(line['path_len_pixel'])),width=2)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['entry_node']],node_color='blue',node_size = 100)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['exit_node']],node_color='red',node_size = 100)


    labels = {line['entry_node']: line['entry_node']}
    nx.draw_networkx_labels(g, pos,labels, font_size=20, font_color='orange')

    labels = {line['exit_node']: line['exit_node']}
    nx.draw_networkx_labels(g, pos,labels, font_size=20,font_color='black')



im = axes.imshow(img_aligned, cmap='gray')

plt.axis("on")
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)


fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes)

# plt.savefig('shortests_paths_crack.jpg',bbox_inches='tight')

plt.show()

# Deprecated

In [ ]:
import networkx as nx
from graph_tool.all import Graph

def nx_to_gt_with_attrs(nx_graph):
    gt_graph = Graph(directed=nx_graph.is_directed())
    vprop_name = gt_graph.new_vertex_property("string")
    eprop_weight = gt_graph.new_edge_property("float")

    nx_to_gt_v = {}
    for node in nx_graph.nodes(data=True):
        v = gt_graph.add_vertex()
        nx_to_gt_v[node[0]] = v
        if 'name' in node[1]:
            vprop_name[v] = str(node[1]['name'])

    for u, v, data in nx_graph.edges(data=True):
        e = gt_graph.add_edge(nx_to_gt_v[u], nx_to_gt_v[v])
        if 'weight' in data:
            eprop_weight[e] = float(data['weight'])

    gt_graph.vertex_properties['name'] = vprop_name
    gt_graph.edge_properties['weight'] = eprop_weight

    return gt_graph

gt_graph = nx_to_gt_with_attrs(g_weighed)

gt.graph_draw(gt_graph, vertex_text=gt_graph.vertex_index, output="wc_cv/cv/two-nodes.png")

# eweight = gt_graph.new_edge_property("weight")
eweight = gt_graph.new_edge_property("float")

vlist = gt_graph.get_vertices()
edges = gt_graph.get_edges()

paths =  list(gt.all_shortest_paths(gt_graph, source=entry_node, target=exit_node, weights=eweight))

# eweight = gt_graph.new_edge_property("weight")
eweight = gt_graph.new_edge_property("float")

vlist = gt_graph.get_vertices()
edges = gt_graph.get_edges()

paths =  list(gt.all_shortest_paths(gt_graph, source=entry_node, target=exit_node, weights=eweight))